In [2]:
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import string
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, accuracy_score, classification_report, confusion_matrix


import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import joblib


In [3]:
df = pd.read_csv("dataproject2025.csv", index_col=0)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1086683 entries, 0 to 1083761
Data columns (total 38 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   issue_d                  1086236 non-null  float64
 1   loan duration            1086236 non-null  float64
 2   annual_inc               1086236 non-null  float64
 3   avg_cur_bal              1086236 non-null  float64
 4   bc_open_to_buy           1086236 non-null  float64
 5   bc_util                  1086236 non-null  float64
 6   delinq_2yrs              1086236 non-null  float64
 7   dti                      1086236 non-null  float64
 8   emp_length               1086236 non-null  object 
 9   emp_title                1086236 non-null  object 
 10  fico_range_high          1086236 non-null  float64
 11  funded_amnt              1086236 non-null  float64
 12  grade                    1086236 non-null  object 
 13  home_ownership           1086236 non-null  obje

In [5]:
df.drop(columns=['Predictions', 'Predicted probabilities'], inplace=True)

In [6]:
df.head()

,issue_d,loan duration,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,dti,emp_length,emp_title,...,pub_rec,pub_rec_bankruptcies,purpose,revol_bal,revol_util,sub_grade,target,tax_liens,zip_code,Pct_afro_american
0,2013.0,0.0,39600.0,1379.0,21564.0,16.1,0.0,2.49,2 years,other,...,0.0,0.0,home_improvement,4136.0,16.1,B2,0.0,0.0,782.0,7.388592
1,2013.0,0.0,55000.0,9570.0,16473.0,53.9,0.0,22.87,10+ years,other,...,0.0,0.0,debt_consolidation,36638.0,61.2,B2,0.0,0.0,481.0,9.745456
2,2013.0,0.0,325000.0,53306.0,13901.0,67.1,0.0,18.55,5 years,sales manager,...,0.0,0.0,debt_consolidation,29581.0,54.6,A3,0.0,0.0,945.0,7.542862
3,2013.0,0.0,130000.0,36362.0,3567.0,93.0,0.0,13.03,10+ years,other,...,0.0,0.0,debt_consolidation,10805.0,67.0,B3,0.0,0.0,809.0,6.598132
4,2013.0,1.0,73000.0,24161.0,4853.0,74.7,1.0,23.13,6 years,other,...,0.0,0.0,debt_consolidation,27003.0,82.8,D5,1.0,0.0,802.0,7.058900


In [7]:
df.isna().sum()

issue_d                  447
loan duration            447
annual_inc               447
avg_cur_bal              447
bc_open_to_buy           447
bc_util                  447
delinq_2yrs              447
dti                      447
emp_length               447
emp_title                447
fico_range_high          447
funded_amnt              447
grade                    447
home_ownership           447
inq_last_6mths           447
int_rate                 447
mo_sin_old_rev_tl_op     447
mo_sin_rcnt_rev_tl_op    447
mo_sin_rcnt_tl           447
mort_acc                 447
mths_since_recent_bc     447
num_actv_bc_tl           447
num_bc_tl                447
num_il_tl                447
num_rev_accts            447
open_acc                 447
pub_rec                  447
pub_rec_bankruptcies     447
purpose                  447
revol_bal                447
revol_util               447
sub_grade                447
target                   447
tax_liens                447
zip_code      

In [8]:
na_rows = df.isna().any(axis=1)
print(f"Number of rows with NA: {na_rows.sum()}")
display(df[na_rows])

print(df.isna().sum(axis=1).value_counts())

Number of rows with NA: 447


,issue_d,loan duration,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,dti,emp_length,emp_title,...,pub_rec,pub_rec_bankruptcies,purpose,revol_bal,revol_util,sub_grade,target,tax_liens,zip_code,Pct_afro_american
2276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1069795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1070263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0     1086236
36        447
Name: count, dtype: int64


In [9]:
# ProfileReport(df)

In [10]:
df_dropped = df.dropna(axis=0) # either y misses or all X miss

In [11]:
def get_features(df: pd.DataFrame()) -> pd.DataFrame():
    """
    Get additional features.
    """        

    df_with_features = (
        df
        .assign(

            # logs --> to money, not months or so
            annual_inc_log = np.log1p(df["annual_inc"]),
            avg_cur_bal_log = np.log1p(df["avg_cur_bal"]),
            fico_range_high_log = np.log1p(df["fico_range_high"]),
            revol_bal_log = np.log1p(df["revol_bal"]),

            # broader zip code area
            zip_code2 = np.round(df["zip_code"] / 10, 0),

            # total balance?
            cur_balance = df["avg_cur_bal"] * df["open_acc"],

            # flags
            delinq_2yrs_flag = df["delinq_2yrs"] >= 1,
            tax_liens_flag = df["tax_liens"] >= 1,

            # shares
            s_actv_bc_tl = df["num_actv_bc_tl"] / (df["open_acc"] + 1e-6),
            s_bc_tl = df["num_bc_tl"] / (df["open_acc"] + 1e-6),
            s_il_tl = df["num_il_tl"] / (df["open_acc"] + 1e-6),
            s_rev_accts = df["num_rev_accts"] / (df["open_acc"] + 1e-6),

            # interactions
            int_rate_x_duration = df["int_rate"] * df["loan duration"], # higher rates are even riskier on 60 vs 36
            dti_x_util = df["dti"] * (df["revol_util"] / 100.0), # debt burden (DTI) is more problematic if utilization of their cards/lines is also high
            revol_bal_income_ratio = df["revol_bal"] / (df["annual_inc"] + 1e-6), # leverage: outstanding revolving balance / income
            fico_x_dti = df["fico_range_high"] * df["dti"], # same DTI can mean different risk depending on FICO score; "do they manage well or not?"
        )
    )

    return df_with_features

df_engineered = get_features(df_dropped)

In [12]:
def categorical_encoding(df: pd.DataFrame) -> pd.DataFrame:
  """Encodings of categorical variables."""

  df_encoded = df.copy()

  # grade to numeric
  grade_map = {c: i+1 for i, c in enumerate(string.ascii_uppercase[:7])}
  df_encoded["grade_num"] = df_encoded["grade"].map(grade_map)

  # sub_grade to numeric
  sg = df_encoded["sub_grade"].astype(str).str.upper().str.strip()
  letter = sg.str[0]
  number = pd.to_numeric(sg.str[1:].str.extract(r"(\d+)", expand=False), errors="coerce")
  letter_map = {ch: i+1 for i, ch in enumerate("ABCDEFG")}
  base = letter.map(letter_map)
  sub_grade_num = (base - 1) * 5 + number
  df_encoded["sub_grade_num"] = sub_grade_num.astype("float32")

  # emp_length to numeric; map prob cleanest; maybe 10+ different?
  emp_length_map = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10
  }

  df_encoded["emp_length_num"] = df_encoded["emp_length"].map(emp_length_map).astype("float32")

  # one-hot
  onehot_cols = ["home_ownership", "purpose", "emp_title"]
  df_encoded = pd.get_dummies(df_encoded, columns=onehot_cols, prefix=onehot_cols, drop_first=True)

  # drop originals
  df_encoded = df_encoded.drop(columns=["grade", "sub_grade", "emp_length"])

  return df_encoded

df_encoded = categorical_encoding(df_engineered)

## STEP 2

In [13]:
target_col = 'target'
# Separate features and target
X = df_encoded.drop(columns=[target_col])
y = df_encoded[target_col]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")
print(f"Target distribution in training set:")
print(y_train.value_counts(normalize=True))
print("\n" + "="*50 + "\n")

# Cross-validation setup
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

print("XGBOOST MODEL")
print("="*50)

# Initialize XGBoost classifier
xgb_model = xgb.XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    verbosity=0
)

# Cross-validation for XGBoost
print("Performing cross-validation...")
xgb_cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=cv, scoring='accuracy')

print(f"XGBoost CV Scores: {xgb_cv_scores}")
print(f"XGBoost CV Mean: {xgb_cv_scores.mean():.4f} (+/- {xgb_cv_scores.std() * 2:.4f})")

# Train XGBoost on full training set
print("Training XGBoost on full training set...")
xgb_model.fit(X_train, y_train)

# Predictions
xgb_train_pred = xgb_model.predict(X_train)
xgb_test_pred = xgb_model.predict(X_test)

# Evaluate XGBoost
xgb_train_acc = accuracy_score(y_train, xgb_train_pred)
xgb_test_acc = accuracy_score(y_test, xgb_test_pred)

print(f"XGBoost Training Accuracy: {xgb_train_acc:.4f}")
print(f"XGBoost Test Accuracy: {xgb_test_acc:.4f}")

print("\nXGBoost Classification Report (Test Set):")
print(classification_report(y_test, xgb_test_pred))


# # Plot confusion matrices
# plt.subplot(2, 2, 2)
# xgb_cm = confusion_matrix(y_test, xgb_test_pred)
# sns.heatmap(xgb_cm, annot=True, fmt='d', cmap='Blues')
# plt.title('XGBoost Confusion Matrix')
# plt.ylabel('True Label')
# plt.xlabel('Predicted Label')

# # Feature importance comparison (top 10 features)
# plt.subplot(2, 2, 4)
# xgb_importance = xgb_model.feature_importances_
# rf_importance = rf_model.feature_importances_

# # Get top 10 features from XGBoost
# top_features_idx = np.argsort(xgb_importance)[-10:]
# top_features = X.columns[top_features_idx]

# x_pos = np.arange(len(top_features))
# plt.barh(x_pos - 0.2, xgb_importance[top_features_idx], 0.4, label='XGBoost', alpha=0.8)
# plt.barh(x_pos + 0.2, rf_importance[top_features_idx], 0.4, label='Random Forest', alpha=0.8)
# plt.yticks(x_pos, top_features)
# plt.xlabel('Feature Importance')
# plt.title('Top 10 Feature Importance Comparison')
# plt.legend()

# plt.tight_layout()
# plt.show()


Training set size: (868988, 112)
Test set size: (217248, 112)
Target distribution in training set:
target
0.0    0.789505
1.0    0.210495
Name: proportion, dtype: float64


XGBOOST MODEL
Performing cross-validation...
XGBoost CV Scores: [0.79460615 0.7945371  0.79438794]
XGBoost CV Mean: 0.7945 (+/- 0.0002)
Training XGBoost on full training set...
XGBoost Training Accuracy: 0.8006
XGBoost Test Accuracy: 0.7951

XGBoost Classification Report (Test Set):
              precision    recall  f1-score   support

         0.0       0.81      0.98      0.88    171518
         1.0       0.57      0.11      0.19     45730

    accuracy                           0.80    217248
   macro avg       0.69      0.55      0.54    217248
weighted avg       0.76      0.80      0.74    217248



In [14]:
import optuna
from sklearn.model_selection import cross_val_score
import xgboost as xgb

# Define the objective function for Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'random_state': 42,
        'eval_metric': 'logloss',
        'verbosity': 0
    }
    
    # Create model with suggested parameters
    model = xgb.XGBClassifier(**params)
    
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    
    # Return mean CV score (Optuna maximizes this)
    return cv_scores.mean()

study = optuna.create_study(direction='maximize', study_name='xgboost_optimization')
study.optimize(objective, n_trials=10)

# Get best parameters
best_params = study.best_params
print(f"\nBest parameters: {best_params}")
print(f"Best CV score: {study.best_value:.4f}")

# Train final model with best parameters
print("\nTraining final optimized XGBoost model...")
optimized_xgb = xgb.XGBClassifier(**best_params)
optimized_xgb.fit(X_train, y_train)

# Evaluate optimized model
optimized_train_pred = optimized_xgb.predict(X_train)
optimized_test_pred = optimized_xgb.predict(X_test)

optimized_train_acc = accuracy_score(y_train, optimized_train_pred)
optimized_test_acc = accuracy_score(y_test, optimized_test_pred)

print(f"Optimized XGBoost Training Accuracy: {optimized_train_acc:.4f}")
print(f"Optimized XGBoost Test Accuracy: {optimized_test_acc:.4f}")

print("\nOptimized XGBoost Classification Report (Test Set):")
print(classification_report(y_test, optimized_test_pred))



[I 2025-09-25 12:11:25,029] A new study created in memory with name: xgboost_optimization
[I 2025-09-25 12:12:33,824] Trial 0 finished with value: 0.7944965871383759 and parameters: {'n_estimators': 160, 'max_depth': 9, 'learning_rate': 0.027628746285086818, 'subsample': 0.9660586550478428, 'colsample_bytree': 0.9479986597300765, 'colsample_bylevel': 0.9821966234200675, 'reg_alpha': 0.02007619755276957, 'reg_lambda': 7.515856292107318e-07, 'min_child_weight': 5}. Best is trial 0 with value: 0.7944965871383759.
[I 2025-09-25 12:13:08,928] Trial 1 finished with value: 0.7929027796281592 and parameters: {'n_estimators': 91, 'max_depth': 10, 'learning_rate': 0.1982029689359225, 'subsample': 0.8954679668559986, 'colsample_bytree': 0.7583013737778234, 'colsample_bylevel': 0.9940873458396307, 'reg_alpha': 0.009726171007469066, 'reg_lambda': 0.05300274571711142, 'min_child_weight': 4}. Best is trial 0 with value: 0.7944965871383759.
[I 2025-09-25 12:13:46,419] Trial 2 finished with value: 0.79


Best parameters: {'n_estimators': 196, 'max_depth': 8, 'learning_rate': 0.06727277923542496, 'subsample': 0.8817326477610854, 'colsample_bytree': 0.8053526658547998, 'colsample_bylevel': 0.9978401763464493, 'reg_alpha': 0.004891250637010883, 'reg_lambda': 0.0032177883420182168, 'min_child_weight': 9}
Best CV score: 0.7954

Training final optimized XGBoost model...
Optimized XGBoost Training Accuracy: 0.8016
Optimized XGBoost Test Accuracy: 0.7953

Optimized XGBoost Classification Report (Test Set):
              precision    recall  f1-score   support

         0.0       0.80      0.98      0.88    171518
         1.0       0.58      0.10      0.17     45730

    accuracy                           0.80    217248
   macro avg       0.69      0.54      0.53    217248
weighted avg       0.76      0.80      0.73    217248



## STEP 9

In [18]:
from joblib import load

optimized_xgb = load('optimized_xgb_model.pkl')

y_pred = optimized_xgb.predict(X_test)

In [19]:
import numpy as np

unique, counts = np.unique(y_pred, return_counts=True)
print(dict(zip(unique, counts)))


{0: 209170, 1: 8078}


In [20]:
print(df.columns)
df["Pct_afro_american"].head()
df["Pct_afro_american"].describe()

# the only ethcnicity variable is Pct_afro_american: Average percentage of African American people living in the area covered by the three-digit ZIP code (census)

Index(['issue_d', 'loan duration', 'annual_inc', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util', 'delinq_2yrs', 'dti', 'emp_length',
       'emp_title', 'fico_range_high', 'funded_amnt', 'grade',
       'home_ownership', 'inq_last_6mths', 'int_rate', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'mths_since_recent_bc', 'num_actv_bc_tl', 'num_bc_tl', 'num_il_tl',
       'num_rev_accts', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies',
       'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'target',
       'tax_liens', 'zip_code', 'Pct_afro_american'],
      dtype='object')


count    1.086236e+06
mean     1.290673e+01
std      1.206943e+01
min      4.301260e-02
25%      3.971440e+00
50%      8.868146e+00
75%      1.853919e+01
max      7.036799e+01
Name: Pct_afro_american, dtype: float64

In [21]:
#splitting the test set by median of this variable

# Extract protected attribute for test set
pct_afro = df.loc[X_test.index, "Pct_afro_american"]

# Median split
median_val = pct_afro.median()
ethnicity_group = pd.Series(
    ["High" if val > median_val else "Low" for val in pct_afro],
    index=pct_afro.index
)

print(ethnicity_group.value_counts())


Low     109416
High    107832
Name: count, dtype: int64


In [26]:
# Creating a new df : results_df
results_df = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_pred,
    "Pct_afro_american": df.loc[X_test.index, "Pct_afro_american"],
    "ethnicity_group": ethnicity_group,
    "annual_inc": df.loc[X_test.index, "annual_inc"],
    "fico_range_high": df.loc[X_test.index, "fico_range_high"],
    "grade": df.loc[X_test.index, "grade"],
    "loan_duration": df.loc[X_test.index, "loan duration"]   # rename to avoid space in column name
})

# ⚠️ Rename column so it has no space
results_df = results_df.rename(columns={"loan_duration": "loan_duration"})

# Quick check
print(results_df.head())
print(results_df["ethnicity_group"].value_counts())

         y_true  y_pred  Pct_afro_american ethnicity_group  annual_inc  \
199648      0.0       0           6.669588             Low    128000.0   
465152      0.0       0           4.657994             Low    128000.0   
1061172     0.0       0          15.194674            High     53000.0   
356750      0.0       0           1.698836             Low     31000.0   
485653      0.0       0           6.522108             Low     70000.0   

         fico_range_high grade  loan_duration  
199648             729.0     B            1.0  
465152             719.0     C            1.0  
1061172            779.0     A            0.0  
356750             674.0     D            0.0  
485653             674.0     D            0.0  
ethnicity_group
Low     109416
High    107832
Name: count, dtype: int64


In [31]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, roc_auc_score

# === 1. Statistical Parity (Demographic Parity)b
dp = results_df.groupby("ethnicity_group")["y_pred"].mean()
print("\nDemographic parity (selection rate):\n", dp)




Demographic parity (selection rate):
 ethnicity_group
High    0.041555
Low     0.032875
Name: y_pred, dtype: float64


In [38]:
# 2. Conditional statistical parity with safe ratio calculation
def conditional_stat_parity_stratified(
    df,
    group_col="ethnicity_group",
    cond_spec={"annual_inc": "q4", "fico_range_high": "q4", "grade": "cat", "loan_duration": "cat"},
    yhat_col="y_pred",
):
    work = df[[group_col, yhat_col] + list(cond_spec.keys())].copy()

    # Bin continuous variables, cast categoricals
    for col, spec in cond_spec.items():
        if spec.startswith("q"):  # e.g. "q4" for quartiles
            q = int(spec[1:])
            work[col] = pd.qcut(work[col], q=q, duplicates="drop")
        elif spec == "cat":
            work[col] = work[col].astype("category")
        else:
            raise ValueError(f"Unknown spec for {col}: {spec}")

    # Selection rate within each stratum and group
    strata = list(cond_spec.keys())
    rates = (
        work.groupby(strata + [group_col])[yhat_col]
            .mean()
            .unstack(group_col)
    )

    # Keep only strata where both groups are present
    rates = rates.dropna()

    # Per-stratum gap and ratio
    gaps = (rates.max(axis=1) - rates.min(axis=1)).rename("gap")
    ratios = (rates.min(axis=1) / rates.max(axis=1)).rename("ratio")

    # Stratum weights (number of samples per stratum)
    weights = work.groupby(strata).size().reindex(gaps.index).rename("n")

    # Weighted gap (always valid)
    w_gap = np.average(gaps.values, weights=weights.values)

    # Weighted ratio (only valid strata, drop NaNs)
    valid_ratios = ratios.dropna()
    if not valid_ratios.empty:
        w_ratio = np.average(valid_ratios.values, weights=weights.loc[valid_ratios.index].values)
    else:
        w_ratio = np.nan

    # Build summary table
    summary = pd.DataFrame({
        "n": weights,
        "gap": gaps,
        "ratio": ratios
    }).sort_values("n", ascending=False)

    return w_gap, w_ratio, rates, summary


# === Run it ===
w_gap, w_ratio, rate_table, per_stratum = conditional_stat_parity_stratified(
    results_df,
    group_col="ethnicity_group",
    cond_spec={
        "annual_inc": "q4",
        "fico_range_high": "q4",
        "grade": "cat",
        "loan_duration": "cat"
    },
    yhat_col="y_pred"
)

print("Weighted gap (max-min across groups within strata):", w_gap)
print("Weighted ratio (min/max across groups within strata, valid strata only):", w_ratio)



Weighted gap (max-min across groups within strata): 0.010311004228228235
Weighted ratio (min/max across groups within strata, valid strata only): 0.4796935594665693


/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/3551137818.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  work.groupby(strata + [group_col])[yhat_col]
/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/3551137818.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weights = work.groupby(strata).size().reindex(gaps.index).rename("n")


In [32]:

# === 3. Equal Opportunity (True Positive Rate per group)
def tpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return tp / (tp + fn) if (tp + fn) > 0 else float("nan")

tpr_values = results_df.groupby("ethnicity_group").apply(
    lambda g: tpr(g["y_true"], g["y_pred"])
)
print("\nTrue positive rate (Equal opportunity):\n", tpr_values)




True positive rate (Equal opportunity):
 ethnicity_group
High    0.111757
Low     0.092008
dtype: float64


/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/3685368568.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tpr_values = results_df.groupby("ethnicity_group").apply(


In [33]:
# === 4. False Positive Rate per group
def fpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return fp / (fp + tn) if (fp + tn) > 0 else float("nan")

fpr_values = results_df.groupby("ethnicity_group").apply(
    lambda g: fpr(g["y_true"], g["y_pred"])
)
print("\nFalse positive rate (Equalized odds):\n", fpr_values)




False positive rate (Equalized odds):
 ethnicity_group
High    0.021919
Low     0.017856
dtype: float64


/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/4251086996.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fpr_values = results_df.groupby("ethnicity_group").apply(


In [34]:
# === 5. Predictive Parity (Precision per group)
prec_values = results_df.groupby("ethnicity_group").apply(
    lambda g: precision_score(g["y_true"], g["y_pred"], zero_division=0)
)
print("\nPrecision (Predictive parity):\n", prec_values)


Precision (Predictive parity):
 ethnicity_group
High    0.587815
Low     0.566861
dtype: float64


/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/2254656791.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  prec_values = results_df.groupby("ethnicity_group").apply(


In [39]:
import pandas as pd

# Build summary DataFrame
fairness_summary = pd.DataFrame({
    "Statistical Parity (DP)": dp,
    "Equal Opportunity (TPR)": tpr_values,
    "Equalized Odds (FPR)": fpr_values,
    "Predictive Parity (Precision)": prec_values
})

# Compute gaps and ratios for each metric
gaps = fairness_summary.max() - fairness_summary.min()
ratios = fairness_summary.min() / fairness_summary.max()

# Nicely formatted output
print("=== Fairness Metrics by Group ===")
print(fairness_summary.round(4))
print("\n=== Disparities (across groups) ===")
print(pd.DataFrame({"Gap (max-min)": gaps.round(4), "Ratio (min/max)": ratios.round(3)}))

# Conditional SP
print("\n=== Conditional Statistical Parity ===")
print(f"Weighted gap (max-min across strata): {w_gap:.4f}")
print(f"Weighted ratio (valid strata only): {w_ratio:.3f}")


=== Fairness Metrics by Group ===
                 Statistical Parity (DP)  Equal Opportunity (TPR)  \
ethnicity_group                                                     
High                              0.0416                   0.1118   
Low                               0.0329                   0.0920   

                 Equalized Odds (FPR)  Predictive Parity (Precision)  
ethnicity_group                                                       
High                           0.0219                         0.5878  
Low                            0.0179                         0.5669  

=== Disparities (across groups) ===
                               Gap (max-min)  Ratio (min/max)
Statistical Parity (DP)               0.0087            0.791
Equal Opportunity (TPR)               0.0197            0.823
Equalized Odds (FPR)                  0.0041            0.815
Predictive Parity (Precision)         0.0210            0.964

=== Conditional Statistical Parity ===
Weighted gap (max-